#Computational Biology Project for CHE-293(Applied Bioinformatics) at San Jose State University

To study switches (proteins with flexible sheaths that “switch” states in response to stimuli) through machine learning techniques and predictive modeling on the Bondugula dataset

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from pandas import read_csv
import numpy as np
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve,  accuracy_score
from sklearn.datasets import make_classification
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import plotly.graph_objects as go
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files_uploaded = files.upload()

In [ ]:
import pandas as pd

data = pd.read_csv("bondugula_JDO_20230125_SLIM (1).csv")
data.columns

Create a column is_switch based on HAS_H, HAS_S, HAS_O, HAS_U.

Adding is_switch column

In [ ]:

raw_data['num_X'] = raw_data['HAS_H'] + raw_data['HAS_S'] + raw_data['HAS_O'] + raw_data['HAS_U']
raw_data['is_switch'] = np.where(raw_data['num_X'] > 1, 1, 0)
percent_switches = (raw_data['is_switch'].sum())/(len(raw_data['is_switch']))
print("Percentage of proteins in dataset that are switches is " + str(round(percent_switches*100,2)) + "%")

Percentage of proteins in dataset that are switches is 16.38%


Removing E6.1, E20.1, E22, ProteinID and Res columns

In [ ]:
data = raw_data.drop(['E6.1', 'E20.1','Res','E22', 'ProteinID'], axis=1)
data.columns

Data Preprocessing

In [ ]:
def normalize_column(quant_column):
    magnitude = np.sqrt((np.power(quant_column,2)).sum())
    return quant_column/magnitude

def standardize_column(quant_column):
    U = np.mean(quant_column)
    o = np.std(quant_column)
    return (quant_column - U)/o

def squarevalue (data,quant_column):
    squared_column=data[quant_column]**2
    return squared_column

def scaling (data,quant_column):
    min_val=data[quant_column].min()
    max_val=data[quant_column].max()
    scaled_column=(data[quant_column]-min_val)/((max_val)-(min_val))
    return scaled_column

In [ ]:
processed_data = data.copy()
processed_data['squared_E6']=squarevalue(data,'E6')
processed_data['squared_E20']=squarevalue(data,'E20')
processed_data['squared_isUnstruct']=squarevalue(data,'isUnstruct')
processed_data['squared_Vkbat']=squarevalue(data,'Vkbat')
processed_data['scaledE6']=scaling(data,'E6')
processed_data['scaledE20']=scaling(data,'E20')
processed_data['scaled_isUnstruct']=scaling(data,'isUnstruct')
processed_data['scaled_Vkbat']=squarevalue(data,'Vkbat')
#processed_data.head(20)

Converting Residue data to numeric values

In [ ]:
processed_data['Res_numeric'] = processed_data['Residue'].apply(lambda x: ord(str(x)))
processed_data['squared_Res_numeric']=squarevalue(data,'Res_numeric')
processed_data['scaled_Res_numeric']=scaling(data,'Res_numeric')
#processed_data['Res_numeric'] = standardize_column(processed_data['Res_numeric'])

One Hot Encoder

In [ ]:
ohe= OneHotEncoder()
ohe_features_array = ohe.fit_transform(processed_data[['chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred']]).toarray()
feature_labels = ohe.categories_

feature_labels = ohe.get_feature_names_out(['chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred'])
print(feature_labels)

ohe_df = pd.DataFrame(ohe_features_array, columns = feature_labels)
#ohe_df.tail(10)

In [ ]:
processed_data = processed_data.reset_index()

In [ ]:
df_mo = pd.concat([processed_data, ohe_df],axis=1)
df_mo = df_mo.drop(columns=['index'])
df_mo=df_mo.drop(columns=['Unnamed: 0'])
df_mo=df_mo.dropna()  #regression doesn't work with empty cells. Have to remove.
df_mo.head(20)

Checking if there are empty cells

In [ ]:
empty_cell_count=df_mo.isnull().sum()
print("empty cell count in column:")
print(empty_cell_count)

#Random Forest


Fitting with true data

Inclusion of 'HAS_H','HAS_S','HAS_O','HAS_U' will give the score 1.0 since they are the true values.

In [ ]:
simple_quant_features = ['E20','E6','isUnstruct','Vkbat', 'HAS_H','HAS_S','HAS_O','HAS_U']

target = "is_switch"

In [ ]:
train_data, test_data = train_test_split(df_mo, test_size=0.25, random_state=1)
X_train = train_data[simple_quant_features]
y_train = train_data[target]
X_test = test_data[simple_quant_features]
y_test = test_data[target]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 25, oob_score = True)
rf_model.fit(train_data[simple_quant_features], train_data[target])
rf_model.get_params()

In [ ]:
print(f"Out of bag estimate score: {rf_model.oob_score_}")
print(f"Train Score: {rf_model.score(train_data[simple_quant_features], train_data[target])}")
print(f"Test Score: {rf_model.score(test_data[simple_quant_features], test_data[target])}")

Modeling with processed data

In [ ]:
feature_sets = [
  ['squared_E20','HAS_H','HAS_S','HAS_U','HAS_O'],
  ['E6','isUnstruct','Vkbat'],
  ['E6','isUnstruct'],
  ['E6','Res_numeric','isUnstruct','Vkbat'],
  ['E6','Res_numeric','isUnstruct'],
  ['E6','Res_numeric','Vkbat'],
  ['E6','Res_numeric'],
  ['E6','Vkbat'],
  ['E6'],
  ['E20','isUnstruct','Vkbat'],
  ['E20','isUnstruct'],
  ['E20','Res_numeric','isUnstruct','Vkbat'],
  ['E20','Res_numeric','isUnstruct'],
  ['E20','Res_numeric','Vkbat'],
  ['E20','Res_numeric'],
  ['E20','Vkbat'],
  ['E20'],
  ['isUnstruct','Res_numeric'],
  ['isUnstruct','Vkbat'],
  ['isUnstruct'],
  ['Res_numeric','isUnstruct','Vkbat'],
  ['Res_numeric'],
  ['Vkbat','Res_numeric'],
  ['Vkbat'],
  ['squared_E6','scaledE6','squared_E20','scaledE20'],
  ['squared_E6','scaledE6'],
  ['squared_E20','scaledE20'],
  ['squared_E6','squared_Vkbat'],
  ['squared_E20','squared_Vkbat'],
  ['scaled_Vkbat'],
  ['scaled_isUnstrct'],
  ['squared_isUnstruct','scaled_isUnstruct']
 ]

In [ ]:
rf_models = []
roc_curves=[]
prob_scores=[]

for i, features in enumerate(feature_sets):
    print(f'{features=}')

    # Fit Random forest
    rf_model = RandomForestClassifier(n_estimators = 25, oob_score = True)
    rf_model.fit(train_data[features], train_data[target])

    print(f"Out of bag estimate score: {rf_model.oob_score_}")
    print(f"Train Score: {rf_model.score(train_data[features], train_data[target])}")
    print(f"Test Score: {rf_model.score(test_data[features], test_data[target])}")

    rf_model_scores = rf_model.predict_proba(test_data[features])[:,1] # extracts probabilities of class 1 (being a switch)
    fpr_reg, tpr_reg, thresholds_reg = roc_curve(test_data[target], rf_model_scores)
    prob_scores.append((rf_model_scores, f"Random Forest {i}"))

In [ ]:
plt.figure(figsize=(8, 8))

for (rfmodel_prob_scores, label) in prob_scores:
    # Compute the ROC curve data points using the correct variable
    fpr_reg, tpr_reg, thresholds_reg = roc_curve(test_data[target], rfmodel_prob_scores)
    # Plot the ROC curve for this model
    plt.plot(fpr_reg, tpr_reg, label=f"{label} (AUC = {roc_auc_score(test_data[target], rfmodel_prob_scores):.2f})")

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Move the legend outside the plot to avoid covering the ROC curves
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.title('Receiver Operating Characteristic (ROC) Curve for Random Forest Models')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')

# Show the plot with all ROC curves
plt.show()

# Logistic Regression


Fitting with true data

In [ ]:
true_logistic_model = LogisticRegression().fit(train_data[simple_quant_features], y_train)

In [ ]:
preds = true_logistic_model.predict(X_test)
prob_preds = true_logistic_model.predict_proba(X_test)

In [ ]:
[print(f"{feat}: {coef:.4f}") for feat, coef in zip(simple_quant_features, true_logistic_model.coef_[0])];
print(f"Intercept: {true_logistic_model.intercept_[0]}")
print(f"Error: {(preds - y_test).sum()}")
print(f"Score: {true_logistic_model.score(X_test, y_test)}")

In [ ]:
With processed data

In [ ]:
logistic_models = []
roc_curves=[]
prob_scores=[]
for i, features in enumerate(feature_sets):
    print(f'{features=}')

    # Fit Logistic Regression model
    logistic_model = LogisticRegression().fit(train_data[features], train_data[target])

    # Print coefficients and intercept
    [print(f"{feature}: {coefficient:.4f}") for feature, coefficient in zip(features, logistic_model.coef_[0])]
    print(f"Intercept: {logistic_model.intercept_[0]:.4f}")

    score = logistic_model.score(test_data[features], test_data[target])
    print(f"Score: {score:.6f}")
    print('')

    auc = np.round(roc_auc_score(test_data[target], logistic_model.predict(test_data[features])), 4)
    print("Auc for our sample data is {}".format(auc))
    print("="*300)
    print('')

    logmodel_prob_scores = logistic_model.predict_proba(test_data[features])[:, 1]
    prob_scores.append((logmodel_prob_scores, f"Logistic Regression {i}"))

In [ ]:
# Checking count in test data for is_switch
sum(test_data[target]  == 0) / len(test_data[target])

In [ ]:
plt.figure(figsize=(8, 8))
for (logmodel_prob_scores, label) in prob_scores:
    fpr, tpr, _ = roc_curve(test_data[target], logmodel_prob_scores)
    roc_curves.append((fpr, tpr, label))
    plt.plot(fpr, tpr, label=f"{label} (AUC = {roc_auc_score(test_data[target], logmodel_prob_scores):.2f})")

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.title('Receiver Operating Characteristic (ROC) Curve for Logistic Regression Models')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.show()

# SVM Model

In [ ]:
svm_models = []
roc_curves_svm = []
prob_scores_svm = []

for i, features in enumerate(feature_sets):
    print(f'{features=}')


    svm_model = LinearSVC().fit(train_data[features], train_data[target])


    score = svm_model.score(test_data[features], test_data[target])
    print(f"Score: {score:.6f}")
    print('')

    auc_svm = np.round(roc_auc_score(test_data[target], svm_model.decision_function(test_data[features])), 3)
    print("AUC for our sample data is {}".format(auc_svm))
    print("=" * 300)
    print('')

    # Store decision function values for ROC curve
    svm_decision_function = svm_model.decision_function(test_data[features])
    prob_scores_svm.append((svm_decision_function, f"Linear SVM {i}"))

# Plot ROC curves for all Linear SVM models
plt.figure(figsize=(8, 8))
for (svm_decision_function, label) in prob_scores_svm:
    fpr, tpr, _ = roc_curve(test_data[target], svm_decision_function)
    roc_curves_svm.append((fpr, tpr, label))
    plt.plot(fpr, tpr, label=f"{label} (AUC = {roc_auc_score(test_data[target], svm_decision_function):.2f})")

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.title('Receiver Operating Characteristic (ROC) Curve for Linear SVM Models')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.legend()
plt.show()